# Data Wrangling for Santa Clara Master Gardener Club by Michelle Petersen

In [199]:
# Imports
%matplotlib inline
import pandas as pd
import numpy as np
import json
import geojson
import shapefile
import requests
from bs4 import BeautifulSoup
import difflib
#import geopandas as gpd
import pprint as pp
import six.moves.urllib as urllib
import matplotlib.pyplot as plt
import re
from timeit import default_timer as timer

## Gather

In [ ]:
df = gpd.read_file('ZillowNeighborhoods-CA/ZillowNeighborhoods-CA.shp')
print(df)

In [ ]:
df.to_csv('California neighborhoods.csv')

In [ ]:
sc_df = df[df['County'] == 'Santa Clara']
print(sc_df)

In [ ]:
sc_df.to_csv('Santa Clara neighborhoods.csv')

In [ ]:
sc_df.plot()

## Assess

In [ ]:
import gspread
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession

# The scope to read the spreadsheet
_GS_SCOPE = ['https://www.googleapis.com/auth/spreadsheets.readonly',
            'https://www.googleapis.com/auth/drive']

# The ID and range of the MasterGardener spreadsheet.
_GS_ID = '1pPOuRZ9YsKwXGKgxzSm9tXCR_dgkCY4WNh3rg_cFX08'

service_account_info = json.load(open('Master Gardener-7e906e507d9d.json'))
credentials = service_account.Credentials.from_service_account_info(
    service_account_info,
    scopes=_GS_SCOPE)

if credentials:
    gc = gspread.Client(auth=credentials)
    gc.session = AuthorizedSession(credentials)
    book = gc.open_by_key(_GS_ID)
    print(book.title)
    worksheet_list = book.worksheets()
    for worksheet in worksheet_list:
        print(worksheet)

In [ ]:
df_contacts = pd.DataFrame(book.get_worksheet(0).get_all_values())
df_contact_neighborhood = pd.DataFrame(book.get_worksheet(1).get_all_values())
df_zillow_neighborhoods = pd.DataFrame(book.get_worksheet(2).get_all_values())
df_nextdoor_neighborhoods = pd.DataFrame(book.get_worksheet(4).get_all_values())

In [ ]:
df_contacts.head()

In [ ]:
df_contacts.reset_index()
df_contacts.columns = ['Name', 'email', 'Residence - Nextdoor Neighborhood', 'Unknown Neighborhood', 'Notes']

In [ ]:
df_contacts = df_contacts[df_contacts['Name'] != 'Name']

In [ ]:
df_contacts['Residence - Nextdoor Neighborhood'] = df_contacts['Residence - Nextdoor Neighborhood'].str.title()

In [ ]:
df_contacts.head()

In [ ]:
df_contact_neighborhood.head()

In [ ]:
df_contact_neighborhood.reset_index()
df_contact_neighborhood.columns = ['Name', 'Nextdoor Neighborhood', 'Unknown Neighborhood']

In [ ]:
df_contact_neighborhood = df_contact_neighborhood[df_contact_neighborhood['Name'] != 'Name']

In [ ]:
df_contact_neighborhood.head()

In [ ]:
df_contact_neighborhood['Nextdoor Neighborhood'] = df_contact_neighborhood['Nextdoor Neighborhood'].str.title()

In [ ]:
df_contact_neighborhood.head()

In [ ]:
df_nextdoor_neighborhoods.head()

In [ ]:
df_nextdoor_neighborhoods.reset_index()
df_nextdoor_neighborhoods.columns = ['State', 'County', 'City', 'Neighborhood', 'Link']

In [ ]:
df_nextdoor_neighborhoods = df_nextdoor_neighborhoods[df_nextdoor_neighborhoods['Neighborhood'] != 'Neighborhood']

In [ ]:
df_nextdoor_neighborhoods.head()

In [ ]:
df_contact_neighborhood.head()

In [ ]:
df_zillow_neighborhoods.head()

In [ ]:
df_zillow_neighborhoods.reset_index()
df_zillow_neighborhoods.columns = ['State', 'County', 'City', 'Neighborhood', 'RegionID', 'Geometry']

In [ ]:
df_zillow_neighborhoods = df_zillow_neighborhoods[df_zillow_neighborhoods['Neighborhood'] != 'Neighborhood']

In [ ]:
df_zillow_neighborhoods.head()

In [ ]:
df_contacts.to_csv('mg_contacts_master.csv', index=False)
df_contact_neighborhood.to_csv('mg_contact_neighborhood_master.csv', index=False)

## Scraping Nextdoor Neighborhoods

state = 'CA'
state_url = 'https://nextdoor.com/find-neighborhood/' + state + '/'

from requests import get
response = requests.get(state_url)
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

hood_city_group = html_soup.find_all('div', class_ = 'hood_group')
print(type(hood_city_group))
print(len(hood_city_group))

df_cities = pd.DataFrame(index=range(5000), columns = ['State', 'City', 'Link'])
df_loc = 0
df_cities.head(1)

for div in hood_city_group:
    city_links = div.findAll('a')
    for a in city_links:
        df_cities.iloc[df_loc, df_cities.columns.get_loc("State")] = 'CA'
        df_cities.iloc[df_loc, df_cities.columns.get_loc("City")] = a.string
        df_cities.iloc[df_loc, df_cities.columns.get_loc("Link")] = a['href'].strip()
        df_loc += 1

df_cities.dropna(axis = 0, inplace=True)
print(df_loc)

df_cities.reindex(columns=['State','County','City', 'Link'])
df_cities.head(1)

df_cities.to_csv('mg_nextdoor_california_cities.csv', index=False)

In [ ]:
cities = ['San Jose','Santa Clara','Sunnyvale', 'Palo Alto','Mountain View',\
          'Cupertino','Milpitas','Los Gatos', 'Gilroy', 'Morgan Hill', 'Campbell',\
          'Los Altos','Saratoga', 'Stanford', 'Los Altos Hills',\
          'San Martin']

df_nextdoor_neighborhoods = pd.DataFrame(index=range(7000), columns = ['State', 'County', 'City', 'Neighborhood', 'Link'])
df_loc = 0
df_nextdoor_neighborhoods.head(1)

for city in cities:
    print(city)
    city_lookup = df_cities[df_cities['City'] == city]["Link"].values[0]
    print(city_lookup)
    response = requests.get(city_lookup)
    responseTxt = response.text.encode('UTF-8')
    html_soup = BeautifulSoup(responseTxt, 'html.parser')
    type(html_soup) 
    hood_neighborhood_group = html_soup.find_all('div', class_ = 'hood_group')
    print(type(hood_neighborhood_group))
    print(len(hood_neighborhood_group))
   
    for div in hood_neighborhood_group:
        neighborhood_links = div.findAll('a')
        for a in neighborhood_links:
            print(city + " " + a.string + " " +  a['href'])
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("State")] = 'CA'
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("County")] = 'Santa Clara'
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("City")] = city
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("Neighborhood")] = str(a.string.encode('UTF-8'))
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("Link")] = str(a['href'].strip())
            df_loc += 1

df_nextdoor_neighborhoods.dropna(axis = 0, inplace=True)
print(df_loc)

df_nextdoor_neighborhoods.head(3)

df_nextdoor_neighborhoods.to_csv('mg_nextdoor_neighborhoods_master.csv', index=False)

In [ ]:
df_contact_neighborhood['Similar Nextdoor Neighborhood'] = 'None'
df_contacts['Similar Nextdoor Neighborhood'] = 'None'

In [ ]:
neighborhoods = list(df_nextdoor_neighborhoods['Neighborhood'])
for index, row in df_contact_neighborhood.iterrows():
    row['Similar Nextdoor Neighborhood'] = difflib.get_close_matches(row['Nextdoor Neighborhood'].lower(), neighborhoods)

In [ ]:
for index, row in df_contacts.iterrows():
    row['Similar Nextdoor Neighborhood'] = difflib.get_close_matches(row['Residence - Nextdoor Neighborhood'].lower(), neighborhoods)

In [ ]:
df_contacts.head(1)

In [ ]:
df_contact_neighborhood.head(1)

In [ ]:
df_contact_neighborhood['Nextdoor Neighborhood'].nunique()

df_contact_neighborhood['Nextdoor Neighborhood'].value_counts()

df_contacts.to_csv('mg_contacts_master.csv', index=False)
df_contact_neighborhood.to_csv('mg_contact_neighborhood_master.csv', index=False)

_GM_API_KEY = '#####'

df_nextdoor_neighborhoods['Similar Google Neighborhood'] = 'None'
df_nextdoor_neighborhoods['Google Place ID'] = 'None'
#df_nextdoor_neighborhoods[Google Location = 'None'

import six.moves.urllib as urllib

for index, row in df_nextdoor_neighborhoods.iterrows():
    neighborhoodquery = urllib.parse.quote(row["Neighborhood"] + ', ' +  row["City"] + ', ' +  row["State"], safe='')
    print(neighborhoodquery)
    response = requests.get('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=' + neighborhoodquery + '&inputtype=textquery&fields=name,id,place_id,plus_code,types,geometry&key=' + _GM_API_KEY)
    resp_json_payload = response.json()
    print(resp_json_payload)                                           
    for k, v in resp_json_payload.items():
        print(k)
        if ((k == 'candidates') & (len(v) > 0)):
            for pair in v:
                if (('neighborhood' == pair['types'][0]) or ('locality' in pair['types'][0])):
                    if ('political' == pair['types'][1]):
                        print(pair['name'])
                        print(pair['place_id'])
                        row["Similar Google Neighborhood"] = pair['name']
                        row["Google Place ID"] = pair['place_id']

df_nextdoor_neighborhoods.to_csv('mg_nextdoor_with_google_master.csv', index=False)

## Parsing geoJSON

In [226]:
# Load the twitter archive data that was downloaded manually from Udacity
df_nextdoor_neighborhoods = pd.read_csv("mg_nextdoor_neighborhood_fulllist.csv")
df_nextdoor_neighborhoods.head(1)

,State,County,City,Neighborhood,Link
0,CA,Santa Clara,San Jose,Albanese Circle,https://nextdoor.com/neighborhood/albanesecirc...


In [227]:
df_nextdoor_neighborhoods["Nextdoor ID"] = 0
df_nextdoor_neighborhoods["Geometry"] = ""
df_nextdoor_neighborhoods["Interests"] = ""
df_nextdoor_neighborhoods["Gardening Interest"] = 0
df_nextdoor_neighborhoods["Percentage of Homeowners"] = 0
df_nextdoor_neighborhoods["Number of Residents"] = 0
df_nextdoor_neighborhoods["Average Age"] = 0
df_nextdoor_neighborhoods['Attributes'] = ""

In [228]:
df_nextdoor_neighborhoods["Link"].fillna("", inplace=True)
df_nextdoor_neighborhoods["Geometry"].fillna("", inplace=True)
df_nextdoor_neighborhoods["Interests"].fillna("", inplace=True)
df_nextdoor_neighborhoods["Attributes"].fillna("", inplace=True)

In [229]:
df_nextdoor_neighborhoods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 13 columns):
State                       1028 non-null object
County                      1028 non-null object
City                        1028 non-null object
Neighborhood                1028 non-null object
Link                        1028 non-null object
Nextdoor ID                 1028 non-null int64
Geometry                    1028 non-null object
Interests                   1028 non-null object
Gardening Interest          1028 non-null int64
Percentage of Homeowners    1028 non-null int64
Number of Residents         1028 non-null int64
Average Age                 1028 non-null int64
Attributes                  1028 non-null object
dtypes: int64(5), object(8)
memory usage: 104.5+ KB


In [230]:
df_nextdoor_neighborhoods.describe()

,Nextdoor ID,Gardening Interest,Percentage of Homeowners,Number of Residents,Average Age
count,1028.0,1028.0,1028.0,1028.0,1028.0
mean,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0


In [231]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

In [232]:
cities = ['San Jose','Santa Clara','Sunnyvale', 'Palo Alto','Mountain View',\
          'Cupertino','Milpitas','Los Gatos', 'Gilroy', 'Morgan Hill', 'Campbell',\
          'Los Altos','Saratoga', 'Stanford', 'Los Altos Hills',\
          'San Martin']

In [233]:
def findAllTheNearbyNeighborhoods(neighborhood_info):
    print("findAllTheNearbyNeighborhoods")
    global df_nextdoor_neighborhoods
    nearbyNeighborhoods = neighborhood_info[neighborhood_info.index('nearbyNeighborhoods: '): neighborhood_info.index('neighborhoodGeometriesJSON:')]
    nearbyNeighborhoods = nearbyNeighborhoods.lstrip()
    nearbyNeighborhoods = nearbyNeighborhoods.rstrip()
    begIndex = len('nearbyNeighborhoods: ')
    nearbyNeighborhoods = nearbyNeighborhoods[begIndex: -1]
    nearby_neighborhood_json = json.loads(nearbyNeighborhoods)
    for nearby_neighborhood in nearby_neighborhood_json:
        if (nearby_neighborhood['city'] in cities):
            nearby_neighborhood['shortName'] = re.sub('\s+', ' ', nearby_neighborhood['shortName']).strip()
            no_entry = df_nextdoor_neighborhoods.loc[df_nextdoor_neighborhoods['Neighborhood'] == nearby_neighborhood['shortName']]
            print(nearby_neighborhood)
            if (len(no_entry) == 0):
                print("Appending: ", nearby_neighborhood)
                df_nextdoor_neighborhoods = df_nextdoor_neighborhoods.append(
                    {"State": nearby_neighborhood['state'],
                     "County": "Santa Clara",
                     "City": nearby_neighborhood['city'],
                     "Neighborhood": nearby_neighborhood['shortName'],
                     "Link": "",
                     "Nextdoor ID": 0,
                     "Geometry": "",
                     "Interests": "",
                     "Gardening Interest": 0,
                     "Percentage of Homeowners": "",
                     "Number of Residents": "",
                     "Average Age": "",
                     "Attributes": ""
                    }, ignore_index=True)
    print("findAllTheNearbyNeighborhoods")

In [234]:
# Iterate over each neighborhood and update the URL and ID
def findAllTheNeighborhoodIDs(neighborhood_info):
    print("findAllTheNeighborhoodIDs")
    global df_nextdoor_neighborhoods
    neighborhoodMap = neighborhood_info[neighborhood_info.index('neighborhoodMapOptions:{'): -1]
    neighborhoodMap = neighborhoodMap.lstrip()
    neighborhoodMap = neighborhoodMap.rstrip()
    #print(neighborhoodMap)
    begURLIndexes = list(find_all(neighborhoodMap, "\"page_url\": \""))  
    endURLIndexes = list(find_all(neighborhoodMap, "\", \"short_name\": ")) 
    print(begURLIndexes)
    print(endURLIndexes)
    begShortNameIndexes = list(find_all(neighborhoodMap, "\"short_name\": \"")) 
    endShortNameIndexes = list(find_all(neighborhoodMap, "\", \"stroke_color\": ")) 
    print(begShortNameIndexes)
    print(endShortNameIndexes)
    gIDIndexes = list(find_all(neighborhoodMap, "\"id\": ")) 
    endIDIndexes = list(find_all(neighborhoodMap, ", \"geometry\": \"{")) 
    print(begIDIndexes)
    print(endIDIndexes)
    if ((len(begURLIndexes) == len(endURLIndexes)) and 
        (len(begShortNameIndexes) == len(endShortNameIndexes)) and 
        (len(begIDIndexes) == len(endIDIndexes))):
        for begShort, endShort, begURL, endURL, begID, endID in zip(begShortNameIndexes, endShortNameIndexes, 
                                                               begURLIndexes, endURLIndexes, 
                                                               begIDIndexes, endIDIndexes):
            print(begShort, endShort)
            print(begURL, endURL)
            print(begID, endID)
            shortname = neighborhoodMap[begShort + len ("\"short_name\": \""): endShort]
            shortname = re.sub('\s+', ' ', shortname).strip()
            print(shortname)
            entry = df_nextdoor_neighborhoods.loc[df_nextdoor_neighborhoods['Neighborhood'] == shortname]
            if (len(entry) == 1):
                pp.pprint(entry)
                index = int(df_nextdoor_neighborhoods.index[df_nextdoor_neighborhoods['Neighborhood'] == shortname][0])
                print(df_nextdoor_neighborhoods.iloc[index])
                if (len(df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Link")]) == 0):
                    page_url = neighborhoodMap[begURL + len ("\"page_url\": \""): endURL]
                    print("Updating Page URL: " + page_url)
                    if (len(page_url) > 0):
                        df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Link")] = page_url
                if (df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] == 0): 
                    neighborhood_id = neighborhoodMap[begID + len ("\"id\": "): endID]
                    print("ID: " + neighborhood_id)
                    df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] = int(neighborhood_id)
                print(df_nextdoor_neighborhoods.iloc[index])
                print("findAllTheNeighborhoodIDs loop")                                
    print("findAllTheNeighborhoodIDs")

In [235]:
# Iterate over each feature and update the row for that neighborhood name
def findAllTheNeighborhoodFeatures(neighborhood_info):
    print("findAllTheNeighborhoodFeatures")
    global df_nextdoor_neighborhoods
    neighborhoodGeometries = neighborhood_info[neighborhood_info.index('neighborhoodGeometriesJSON: '): neighborhood_info.index('neighborhoodMapOptions:{')]
    neighborhoodGeometries = neighborhoodGeometries.lstrip()
    neighborhoodGeometries = neighborhoodGeometries.rstrip()
    begIndex = len('neighborhoodGeometriesJSON: ')
    neighborhoodGeometries = neighborhoodGeometries[begIndex: -1]
    neighborhood_json = json.loads(neighborhoodGeometries)
    for feature in neighborhood_json["features"]:
        props = feature["properties"]
        props["hood_name"] = strip_extra_spaces(props["hood_name"])
        print("Name: %s, ID: %s \n" % (props["hood_name"], props["hood_id"]))
        entry = df_nextdoor_neighborhoods.loc[(df_nextdoor_neighborhoods['Neighborhood'] == props["hood_name"]) and
                                             (df_nextdoor_neighborhoods['Nextdoor ID'] == props["hood_id"])]
        print(entry)
        if (len(entry) == 1):
            index = int(df_nextdoor_neighborhoods.index[df_nextdoor_neighborhoods['Neighborhood'] == props["hood_name"]][0])
            print(df_nextdoor_neighborhoods.iloc[index])
            #Save the feature for that neighborhood without the featurecollection 
            print("Geometry len: " + str(len(df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Geometry")])))
            if (len(df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Geometry")]) == 0):
                feature['properties']['State'] = df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("State")]
                feature['properties']['City'] = df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("City")]
                feature['properties']['County'] = "Santa Clara"
                df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Geometry")] = feature
            if (df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] == 0): 
                df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] = props["hood_id"]
            print(df_nextdoor_neighborhoods.iloc[index])
        print("findAllTheNeighborhoodFeatures loop")
    print("findAllTheNeighborhoodFeatures")

In [236]:
def iterateOneNeighborhood(currentRowIndex):
    if len(df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]) > 0:
        neighborhood_lookup = df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]
        response = requests.get(neighborhood_lookup)
        responseTxt = response.text.encode('UTF-8')
        html_soup = BeautifulSoup(responseTxt, 'html.parser')
     
        try:
            # find the variable with all the neighborhood info
            scripts = html_soup.findAll('script')
            neighborhood_info = scripts[5].string.strip()
            neighborhood_info = neighborhood_info[neighborhood_info.index('['):-1]
        
            findAllTheNearbyNeighborhoods(neighborhood_info)
            pp.pprint(df_nextdoor_neighborhoods.iloc[currentRowIndex])

            findAllTheNeighborhoodIDs(neighborhood_info)
            pp.pprint(df_nextdoor_neighborhoods.iloc[currentRowIndex])

            findAllTheNeighborhoodFeatures(neighborhood_info)
            pp.pprint(df_nextdoor_neighborhoods.iloc[currentRowIndex])

            # Update with the interest info
            try:
                begInterestIndex = neighborhood_info.index('interests: [')
                endInterestIndex = neighborhood_info.index('iosUrl: "https://')
                if ((begInterestIndex > 0) and (endInterestIndex > 0)):
                    interests = neighborhood_info[begInterestIndex: endInterestIndex]
                    interests = interests[interests.index('[')+1: interests.index(']')]
                    interests = interests.replace('"', "")
                    interest_list = interests.split(", ")
                    print(interest_list)
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Interests")] = interest_list
                    if ('Gardening & Landscape' in interest_list):
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Gardening Interest")] = interest_list.index('Gardening & Landscape')
            except:
                print("No interests found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
        
            # Update with the attribute info
            try:
                begAttrIndex = neighborhood_info.index('attributes: [')
                endAttrIndex = neighborhood_info.index('census:')
                if ((begAttrIndex > 0) and (endAttrIndex > 0)):
                    attributes = neighborhood_info[begAttrIndex: endAttrIndex]
                    attributes = attributes[attributes.index('[')+1: attributes.index(']')]
                    attributes = attributes.replace('"', "")
                    attributes_list = attributes.split(", ")
                    print(attributes_list)
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Attributes")] = attributes_list
            except:
                print("No attributes found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
           
            # Update with the census info
            try:
                begCensusIndex = neighborhood_info.index('census: {')
                endCensusIndex = neighborhood_info.index('city: ')
                if ((begCensusIndex > 0) and (endCensusIndex > 0)):
                    census = neighborhood_info[begCensusIndex: endCensusIndex]
                    census = census[census.index('{'): census.index('}')+1]
                    census = json.loads(census)
                    print("Census: %s\n" % str(census))
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Percentage of Homeowners")] = int(census['homeowners'])
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Number of Residents")] = int(census['population'])
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Average Age")] = int(census['age'])
            except:
                print("No census found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
        
        except:
            print("Error parsing")
    return (neighborhood_info)

In [209]:
neighborhood_info = iterateOneNeighborhood(260)

findAllTheNearbyNeighborhoods
{'shortName': 'Boulder', 'hostname': '4boulder', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Capewood', 'hostname': '4capewood', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Cataldi', 'hostname': '4cataldi', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Cherrywood', 'hostname': '4cherrywoodca', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Knights Bridge', 'hostname': '4knightsbridge', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Morrill', 'hostname': '4morrill', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Noble', 'hostname': '4noble', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Ohlone', 'hostname': '4ohlone', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Pinnacle', 'hostname': '4pinnaclesanjose', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Ruskin', 'hostname': '4ruskin', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Sierramont', 'hostname': '4sierramont', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'S

In [212]:
findAllTheNeighborhoodFeatures(neighborhood_info)

findAllTheNeighborhoodFeatures
Name: Piedmont, ID: 170132 

    State       County      City Neighborhood  \
260    CA  Santa Clara  San Jose     Piedmont   
638    CA  Santa Clara  Milpitas     Piedmont   

                                                  Link  Nextdoor ID Geometry  \
260  https://nextdoor.com/neighborhood/4piedmont--s...            0            
638  https://nextdoor.com/neighborhood/piedmontca--...            0            

    Interests  Gardening Interest  Percentage of Homeowners  \
260                             0                         0   
638                             0                         0   

     Number of Residents  Average Age Attributes  
260                    0            0             
638                    0            0             
findAllTheNeighborhoodFeatures loop
Name: Boulder, ID: 170136 

   State       County      City Neighborhood  \
31    CA  Santa Clara  San Jose      Boulder   

                                               

In [214]:
print(neighborhood_info)

[{"agencyName": "San Jos\u00e9 City Council", "hostname": "4piedmont", "subject": "San Jos\u00e9 Clean Energy is LIVE!", "postId": "102055319", "urlSlug": "san-jose-city-council", "avatarPhoto": "https://d3926qxcw0e1bh.cloudfront.net/pages_avatar_photos/fc/92/fc923118a79f09820c9a021fc54a6087.jpg", "groupId": "533104"}, {"agencyName": "Santa Clara Valley Water District", "hostname": "4piedmont", "subject": "Statement from Chair Linda J. LeZotte on storm readiness", "postId": "101984699", "urlSlug": "santa-clara-valley-water-district-1", "avatarPhoto": "https://d3926qxcw0e1bh.cloudfront.net/pages_avatar_photos/01/03/010310b823d05ffa89499baf1d1221f8.jpg", "groupId": "1458946"}, {"agencyName": "San Jos\u00e9 City Council", "hostname": "4piedmont", "subject": "Advancements on my strongest efforts to clean Monterey Road", "postId": "101982974", "urlSlug": "san-jose-city-council", "avatarPhoto": "https://d3926qxcw0e1bh.cloudfront.net/pages_avatar_photos/fc/92/fc923118a79f09820c9a021fc54a6087.

In [202]:
df_nextdoor_neighborhoods["Neighborhood"].nunique()

1031

In [213]:
pp.pprint(df_nextdoor_neighborhoods.iloc[260])

State                                                                      CA
County                                                            Santa Clara
City                                                                 San Jose
Neighborhood                                                         Piedmont
Link                        https://nextdoor.com/neighborhood/4piedmont--s...
Nextdoor ID                                                                 0
Geometry                                                                     
Interests                                                                    
Gardening Interest                                                          0
Percentage of Homeowners                                                    0
Number of Residents                                                         0
Average Age                                                                 0
Attributes                                                      

In [221]:
name = re.sub('\s+', ' ', "Bevin   Brooke ").strip()
print(name)

Bevin Brooke


In [ ]:
df_nextdoor_neighborhoods.head(10)

In [ ]:
df_nextdoor_neighborhoods.tail(10)

In [ ]:
df_nextdoor_neighborhoods.to_csv('mg_nextdoor_withprops_master.csv', index=False)

In [237]:
currentIndex = 0
while True:
    print("Current Index: " + str(currentIndex))
    print("Number of Neighborhoods: " + str(len(df_nextdoor_neighborhoods)))
    iterateOneNeighborhood(currentIndex) 
    currentIndex += 1
    if currentIndex > (len(df_nextdoor_neighborhoods) / 10):
        break

Current Index: 0
Number of Neighborhoods: 1028
findAllTheNearbyNeighborhoods
{'shortName': 'Goodyear/Alma', 'hostname': '3goodyearalma', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Alvin/Burdette', 'hostname': '7alvinburdette', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Bevin Brook', 'hostname': '7bevinbrook', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Garden/Villa Monterey', 'hostname': '7gardenvillamonterey', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Hillsdale', 'hostname': '7hillsdale', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Kennedy', 'hostname': '7kennedy', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Kenwood', 'hostname': '7kenwood', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Lone Bluff', 'hostname': '7lonebluff', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Los Arboles', 'hostname': '7losarboles', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Meadows', 'hostname': '7meadowssanjose', 'city': 'San Jose', 'state': 'CA'}
{'s

findAllTheNearbyNeighborhoods
{'shortName': 'Meadows', 'hostname': '7meadowssanjose', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Stonegate East', 'hostname': '7stonegateeast', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Stonegate West', 'hostname': '7stonegatewest', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Summerside', 'hostname': '7summerside', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Whaley', 'hostname': '7whaley', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Windmill Springs', 'hostname': '7windmillsprings', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Yerba Buena', 'hostname': '7yerbabuena', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Edge', 'hostname': '8edge', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Leyva', 'hostname': '8leyva', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Meadow Fair', 'hostname': '8meadowfair', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Albanese Circle', 'hostname': 'albanesecircle', 'city': '

findAllTheNearbyNeighborhoods
{'shortName': 'Carson', 'hostname': '10carsonsj', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Goodyear/Alma', 'hostname': '3goodyearalma', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Canoas Garden', 'hostname': '6canoasgarden', 'city': 'San Jose', 'state': 'CA'}
Appending:  {'shortName': 'Canoas Garden', 'hostname': '6canoasgarden', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Evans', 'hostname': '6evans', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Farm Drive', 'hostname': '6farmdrive', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Summercreek', 'hostname': '6summercreek', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Willows in San Jose', 'hostname': '6willowsinsanjose', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Mill Pond 1 & 2', 'hostname': '7caroldrive', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Communications Hill (South)', 'hostname': '7communicationhill', 'city': 'San Jose', 'state': 'CA'}
{'shortName':

findAllTheNearbyNeighborhoods
{'shortName': 'Hensley', 'hostname': '3hensley', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Horace Mann', 'hostname': '3horacemann', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Hyde Park', 'hostname': '3hydeparksanjose', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Japantown', 'hostname': '3japantownsanjose', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Modern Ice', 'hostname': '3modernice', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Northside', 'hostname': '3northside', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Rosemary Gardens', 'hostname': '3rosemarygardens', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Vendome - Civic Center', 'hostname': '3vendomesj', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'California Gateway', 'hostname': '4californiagateway', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'McKay-Ringwood', 'hostname': '4mckayringwood', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'North Flea

findAllTheNearbyNeighborhoods
{'shortName': 'Brooktree', 'hostname': '4brooktree', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'California Gateway', 'hostname': '4californiagateway', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Capewood', 'hostname': '4capewood', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Cataldi', 'hostname': '4cataldi', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Cherrywood', 'hostname': '4cherrywoodca', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Countrybrook/Lagoon', 'hostname': '4countrybrooklagoon', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Courtyards/Villas at Lundy', 'hostname': '4courtyardsvillas', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Flickinger North', 'hostname': '4flickingernorth', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Gordy', 'hostname': '4gordy', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Knights Bridge', 'hostname': '4knightsbridge', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'McK

...

findAllTheNearbyNeighborhoods
{'shortName': 'East Ct West Ct', 'hostname': '3eastctwestct', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'El Rancho Verde', 'hostname': '3elranchoverde', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Anne Darling - Little Portugal North', 'hostname': '3littleportugalnorth', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Northside', 'hostname': '3northside', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Pine Hollow', 'hostname': '3pinehollow', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Roosevelt', 'hostname': '3roosevelt', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Wooster/Tripp', 'hostname': '3woostertripp', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Commodore', 'hostname': '4commodore', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Creekland', 'hostname': '4creekland', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Pacheco', 'hostname': '4pacheco', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Ponderosa Woo

findAllTheNearbyNeighborhoods
{'shortName': 'Almaden Meadows', 'hostname': '10almadenmeadows', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Almaden Winery', 'hostname': '10almadenwinery', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Camden Village', 'hostname': '10camdenvillage', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Croydon', 'hostname': '10croydonsanjose', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Guadalupe Oak Grove', 'hostname': '10guadalupeoakgrove', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Holland', 'hostname': '10holland', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Jeffrey Fontana Park', 'hostname': '10jeffreyfontanaparkarea', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Los Alamitos', 'hostname': '10losalamitos', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Montevideo', 'hostname': '10montevideo', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Oak Canyon', 'hostname': '10oakcanyon', 'city': 'San Jose', 'state': 'CA'}
{'

findAllTheNearbyNeighborhoods
{'shortName': 'Checkers', 'hostname': '3checkers', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'El Rancho Verde', 'hostname': '3elranchoverde', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Pine Hollow', 'hostname': '3pinehollow', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Commodore', 'hostname': '4commodore', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Creekland', 'hostname': '4creekland', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Flickinger South', 'hostname': '4flickingersouth', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Ohlone', 'hostname': '4ohlone', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Pacheco', 'hostname': '4pacheco', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Penitencia', 'hostname': '4penitencia', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Ponderosa Woods', 'hostname': '4ponderosawoods', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Rymar', 'hostname': '4rymar', 'city': 'San Jose'

findAllTheNearbyNeighborhoods
{'shortName': 'Berry Park', 'hostname': '10berrypark2', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Blossom Hill Homes', 'hostname': '10blossomhillhomes', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Blossom Village', 'hostname': '10blossomvillage', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Cahalan', 'hostname': '10cahalan', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Canoas East', 'hostname': '10canoaseast', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Canoas West', 'hostname': '10canoaswest', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Carson', 'hostname': '10carsonsj', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Comanche', 'hostname': '10comanche', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Del Robles', 'hostname': '10delrobles', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Hillview', 'hostname': '10hillviewsanjose', 'city': 'San Jose', 'state': 'CA'}
{'shortName': 'Mc Kuen', 'hostname': '10mckuen', 

KeyboardInterrupt: 

### Quality

##### Data Quality Dimensions
Completeness: Do we have all of the records that we should? Do we have missing records or not? Are there specific rows, columns, or cells missing? 

Validity: We have the records, but they're not valid, i.e., they don't conform to a defined schema. A schema is a defined set of rules for data. These rules can be real-world constraints (e.g. negative height is impossible) and table-specific constraints (e.g. unique key constraints in tables). 

Accuracy: Inaccurate data is wrong data that is valid. It adheres to the defined schema, but it is still incorrect. Example: a patient's weight that is 5 lbs too heavy because the scale was faulty. 

Consistency: Inconsistent data is both valid and accurate, but there are multiple correct ways of referring to the same thing. Consistency, i.e., a standard format, in columns that represent the same data across tables and/or within tables is desired. 

##### `df_tweets` Table Actions 
- Blah Blah
##### `df_tweet_additional_info` Table Actions
- Remove retweets.
- 

### Tidiness
##### Tidiness Criteria:
- Each variable you measure should be in one column.
- Each different observation of that variable should be in a different row.
- There should be one table for each “kind” of variable.
- Related tables should each include a column that allows them to be linked.
##### Tidiness Actions
- 

## Clean

##### Define
Remove tweets from the df_tweets and df_tweet_additional_info tables that do not have images in the df_image_predict table.

##### Code

##### Test

## Storing

Store the clean DataFrame in a CSV file named SantaClaraMasterGardener_master.csv.

## Analysis and Visualization

### Viz 1: